In [2]:
import os
%pwd

'/home/towet/Desktop/Visions/OpenProjects/Vegetable-Type-Classification/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/towet/Desktop/Visions/OpenProjects/Vegetable-Type-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    train_ds: Path
    val_ds: Path
    test_ds: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    height: int
    width: int
    num_classes: int

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

2024-01-23 16:02:15.508781: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 16:02:19.250226: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 16:02:28.204345: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        params = self.params
        train_ds = tf.keras.utils.image_dataset_from_directory(os.path.join(self.config.data_ingestion.unzip_dir, "Vegetable Images", "train"), shuffle=True)
        val_ds = tf.keras.utils.image_dataset_from_directory(os.path.join(self.config.data_ingestion.unzip_dir, "Vegetable Images", "validation"), shuffle=True)
        test_ds = tf.keras.utils.image_dataset_from_directory(os.path.join(self.config.data_ingestion.unzip_dir, "Vegetable Images", "test"), shuffle=True)
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            train_ds=list(train_ds),
            val_ds = list(val_ds),
            test_ds = list(test_ds),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            height=params.height,
            width=params.width,
            num_classes=params.CLASSES
        )

        return training_config

In [8]:
import tensorflow as tf

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    

    
    
    # Perform Data Processing on the train, val, test dataset
    def train(self):
        data_preprocess = tf.keras.Sequential(
        name="data_preprocess",
        layers=[
            tf.keras.layers.Resizing(self.config.height, self.config.width), # Shape Preprocessing
            tf.keras.layers.Rescaling(1.0/255), # Value Preprocessing
        ]
        )
        train_ds = self.config.train_ds.map(lambda x, y: (data_preprocess(x), y))
        val_ds = self.config.val_ds.map(lambda x, y: (data_preprocess(x), y))
        test_ds = self.config.test_ds.map(lambda x, y: (data_preprocess(x), y))
        

        pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=[self.config.height,self.config.width, 3])
        pretrained_model.trainable=False
        vgg16_model = tf.keras.Sequential([
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(self.config.num_classes, activation='softmax')
        ])
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint("final_model.h5", save_best_only=True)

        early_stopping_callback = tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",patience=5, restore_best_weights=True
        )

        vgg16_model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
        history_vgg16 = vgg16_model.fit(train_ds, epochs=self.config.EPOCHS, validation_data=val_ds,callbacks=[checkpoint_callback,early_stopping_callback])
        vgg16_model.save(self.config.trained_model_path)

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.train()
    
except Exception as e:
    raise e

[2024-01-23 16:03:05,957: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-23 16:03:06,098: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-23 16:03:06,100: INFO: common: created directory at: artifacts]
Found 15000 files belonging to 15 classes.
Found 3000 files belonging to 15 classes.
Found 3000 files belonging to 15 classes.
[2024-01-23 16:03:09,305: INFO: common: created directory at: artifacts/training]


2024-01-23 16:03:09.519580: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [15000]
	 [[{{node Placeholder/_4}}]]
2024-01-23 16:03:09.523198: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [15000]
	 [[{{node Placeholder/_4}}]]


KeyboardInterrupt: 